In [2]:
import mnist_loader
import numpy as np

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
print len(training_data)

50000


In [9]:
def sigmoid(z): # sigmoid function
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z): # sigmoid prime
    return sigmoid(z)*(1.0-sigmoid(z))

class Network:
    def __init__(self, layer_sizes):
        self.layers = []
        self.weights = []
        self.biases = []
        self.layers = len(layer_sizes)
        for input_size, output_size in zip(layer_sizes[:-1], layer_sizes[1:]):
            weights = np.random.normal(size=(output_size, input_size))
            self.weights.append(weights)
            bias = np.random.normal(size=(output_size, 1))
            self.biases.append(bias)
            
    def forward(self, x):
        a = x
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, a) + b
            a = sigmoid(z)
        return a
    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.forward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def forward_backprop(self, x, y):
        a = x
        zs = []
        activations = [x]
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, a) + b
            zs.append(z)
            a = sigmoid(z)
            activations.append(a)
        errors = []
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        delta = (activations[-1]-y)*sigmoid_prime(zs[-1]) # final detal_L
        nabla_b[-1] = delta #important
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #important
        for l in range(2, self.layers):
            z = zs[-l]
            delta = np.dot(self.weights[-l+1].transpose(), delta) * \
                sigmoid_prime(z)
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (activations, nabla_w, nabla_b)
    
n = input_size = len(training_data[0][0])    
network = Network([n, 30, 10])
epoc = 0
max_epoc = 1
learning_rate = 0.01
while epoc < max_epoc:
    epoc += 1
    error = 0
    nabla_ws = []
    nabla_bs = []
    for x, y in training_data:
        activations, nabla_w, nabla_b = network.forward_backprop(x, y)
        nabla_ws.append(nabla_w)
        nabla_bs.append(nabla_b)
    delta_w = []
    delta_b = []
    for i in range(network.layers-1):
        delta_w.append(sum([w[i] for w in nabla_ws]))
        delta_b.append(sum([b[i] for b in nabla_bs]))
    for i in range(network.layers-1):
        network.weights[i] = network.weights[i] - learning_rate*delta_w[i]
        network.biases[i] = network.biases[i] - learning_rate*delta_b[i]
    
    for x, y in training_data:
        activations, nabla_w, nabla_b = network.forward_backprop(x, y)
    print(network.evaluate(test_data))

1032
